MOUNT


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Enable GPU

In [ ]:
!pip install transformers datasets


Load the dataset

In [ ]:
!pip install --upgrade pip
!pip install datasets --force-reinstall

In [1]:
import datasets
print(datasets.__version__)  # Should print the version number if installed successfully


3.2.0


In [1]:
from datasets import load_dataset

dataset = load_dataset("SKNahin/bengali-transliteration-data")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


SPLIT the dataset

In [2]:
train_test_split = dataset['train'].train_test_split(test_size=0.2)  # 80/20 split
train_data = train_test_split['train']
val_data = train_test_split['test']

Load tokenizer

In [3]:
from transformers import AutoTokenizer

#for mt5-small
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [7]:
print(dataset['train'].column_names)


['bn', 'rm']


The Columns are

BN - BAngla
RM - Banglish

In [31]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["rm"],
        max_length=128,
        padding="max_length",
        truncation=True,
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["bn"],
            max_length=128,
            padding="max_length",
            truncation=True,
        )

    model_inputs["labels"] = labels["input_ids"]

    labels_with_ignore_index = []
    for labels_example in labels["input_ids"]:
        labels_example = [label if label != tokenizer.pad_token_id else -100 for label in labels_example]
        labels_with_ignore_index.append(labels_example)

    model_inputs["labels"] = labels_with_ignore_index
    return model_inputs

tokenized_train_data = train_data.map(preprocess_function, batched=True)
tokenized_val_data = val_data.map(preprocess_function, batched=True)


Map:   0%|          | 0/4004 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1002 [00:00<?, ? examples/s]

TEST

In [32]:
print(tokenized_train_data[0])
print(tokenized_val_data[0])

{'bn': 'ডুয়াল অ্যাপ টা কাজ করতেছে না কেনন ?', 'rm': 'Dual app ta kaj kortece na kenn ?', 'input_ids': [24624, 259, 802, 576, 1357, 47702, 541, 294, 122930, 259, 291, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [21612, 48458, 2369, 33033, 7022, 259, 3459, 3795, 3279, 2212, 1573,

Necessary dependency for the model

In [ ]:
!pip install --upgrade transformers
!pip install torch
!pip install tensorflow
!pip install sentencepiece


Load the Pre-trained Model

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")


# FINE TUNE OUR MODEL

# FINE TUNING AGAIN

In [45]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Model and tokenizer reloaded and ready!")


/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Model and tokenizer reloaded and ready!


In [28]:
!pip install sacrebleu

In [74]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Model and tokenizer reloaded and ready!")

from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
training_args = Seq2SeqTrainingArguments(
    output_dir="./mt5-banglish-bangla",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    predict_with_generate=True,
    save_strategy="epoch",
    logging_steps=100,
    gradient_accumulation_steps=4,
    warmup_ratio=0.1,
    fp16=torch.cuda.is_available() if torch.cuda.is_available() else False
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_val_data,
    tokenizer=tokenizer,
    # Removed compute_metrics
)



/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Model and tokenizer reloaded and ready!


<ipython-input-74-54ac48e01575>:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [75]:
trainer.train()

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`labels` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

# TESTING

In [54]:

input_text = "ami bhalo achi"


inputs = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True)
inputs = {key: value.to(device) for key, value in inputs.items()}  # Move to the same device as the model

outputs = model.generate(**inputs)


translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Translated text:", translated_text)


Translated text: <extra_id_0> ompartmentali <extra_id_1> a dlotesi uli di ilani , bili ami
